In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
from PyPDF2 import PdfReader

file_path = "The-magic-of-thinking-big.pdf"
reader = PdfReader(file_path)

output_file_path = "Book.txt"
with open(output_file_path, "w", encoding="utf-8") as output_file:

    for page_number, page in enumerate(reader.pages, start=1):
        text = page.extract_text()
        output_file.write(f"--- Page {page_number} ---\n")
        output_file.write(text)
        output_file.write("\n\n")
print(f"Text extracted and saved to {output_file_path}")

In [3]:
with open('Book.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [ ]:
total_words

In [6]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list) + 1):  
        n_gram_sequence = token_list[:i]
        input_sequences.append(n_gram_sequence)

In [ ]:
print("input_sequences:", input_sequences)

In [14]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
max_sequence_len

In [ ]:
input_sequences

In [10]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [11]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)

In [15]:
history = model.history.history

In [18]:
accuracy = history['accuracy']
loss = history['loss']

In [20]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(range(100), accuracy, label='Training Accuracy')
plt.legend(loc='lower right')
plt.title('Training Accuracy')

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 2)
plt.plot(range(100), loss, label='Training Loss')
plt.legend(loc='upper right')
plt.title('Training Loss')

In [ ]:
seed_text = "how to think"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

In [ ]:
model_version = 1
model.save(f"Models/{model_version}.h5")

In [35]:
model_version = 1
model.save(f"Models/{model_version}.keras")

In [ ]:
model_version = 1
model.export(f"Models/{model_version}")

In [16]:
import pickle
with open('tokenizer.pkl', 'wb') as file:
    pickle.dump(tokenizer, file)